In [2]:
import pickle
import lzma

with lzma.open('movie_data_movielens.xz', 'rb') as f:
    train_movie_ratings_list, test_movie_ratings_list = pickle.load(f)

In [2]:
with lzma.open('params_k=10_2024-05-14 18:52:32.259848.xz', 'rb') as f:
    params = pickle.load(f)

In [3]:
movie_ratings_list = [a+b for a, b in zip(train_movie_ratings_list, test_movie_ratings_list)]
del train_movie_ratings_list, test_movie_ratings_list

In [1]:
with lzma.open('movies_map.xz', 'rb') as f:
    title_to_id_moviesmap, id_to_title_moviesmap, id_to_genres_moviesmap = pickle.load(f)

NameError: name 'lzma' is not defined

In [5]:
with lzma.open('general_movielens.xz', 'rb') as f:
    userid_to_index, index_to_userid, movieid_to_index, index_to_movieid = pickle.load(f)

In [6]:
import numpy as np
unpopular_movies = []
# movie_indices = []
for index, movie in enumerate(movie_ratings_list):
    if len(movie) < 50:
        unpopular_movies.append(index)
unpopular_movies = np.array(unpopular_movies)

In [7]:
locals().update(params)

In [8]:
import numpy as np

def train_new_user(movies_list, iter=400):
    U = np.zeros((k))

    for _ in range(iter):
        movies_array = np.array(movies_list)
        ratings = movies_array[:, 1]
        indices = movies_array[:, 0].astype(int)
        user_bias = lmb * np.sum(ratings - np.einsum('i,ji->j', U, V[indices]) - item_biases[indices]) / (lmb * len(indices) + gamma)
    
        left = np.sum(np.einsum('ij,il->ijl', V[indices], V[indices]), axis=0)
        right = np.sum(np.einsum('ji,j->ji', V[indices], ratings - user_bias - item_biases[indices]), axis=0)
    
        U = np.linalg.solve(lmb * left + tau * np.eye(k), lmb * right)
        # print(U, user_bias)
    return U

In [9]:
def get_valid_movies(movie_indices, n):
    valid_movies = []
    for movie in movie_indices:
        if movie not in unpopular_movies:
            valid_movies.append(movie)
    return valid_movies[:n]

In [10]:
def predict(movies_list, n=10, threshold=0.1):
    movie_ids = []
    U = train_new_user(movies_list)
    movie_indices = np.argsort(np.einsum('i,ji->j', U, V) + threshold*item_biases)[::-1]
    predictions = get_valid_movies(movie_indices, n)
    # predictions = movie_indices[:n]
    for i, indices in enumerate(predictions):
        movie_id = index_to_movieid[indices]
        movie_ids.append(movie_id)
        print(f'{i+1}:{id_to_title_moviesmap[movie_id]}')
    # return movie_ids

In [11]:
import re
def check(val, movies):
  matched = []
  for movie in movies:
    if re.search(val, movie.lower()):
      matched.append(movie)
  return matched

In [12]:
def get_possible_movie_indices(search_movie, movies_map):
  possible_movies = movies_map.keys()
  for val in search_movie.split():
    possible_movies = check(val, possible_movies)

  for possible_movie in possible_movies:
    try:
        print(f'{movieid_to_index[movies_map[possible_movie]]} : {possible_movie} : {movies_map[possible_movie]}')
    except KeyError:
        continue

In [15]:
get_possible_movie_indices('lord of the rings', title_to_id_moviesmap)

2189 : lord of the rings, the (1978) : 2116
202 : lord of the rings: the fellowship of the ring, the (2001) : 4993
36 : lord of the rings: the two towers, the (2002) : 5952
227 : lord of the rings: the return of the king, the (2003) : 7153
58405 : the lord of the g-strings: the femaleship of the string (2003) : 182439


In [16]:
predict([(2189, 5)], 20, 0.2)

1:lord of the rings, the (1978)
2:lord of the rings: the return of the king, the (2003)
3:lord of the rings: the two towers, the (2002)
4:the hobbit: the battle of the five armies (2014)
5:lord of the rings: the fellowship of the ring, the (2001)
6:hobbit: the desolation of smaug, the (2013)
7:star wars: episode iii - revenge of the sith (2005)
8:star wars: episode ii - attack of the clones (2002)
9:hobbit: an unexpected journey, the (2012)
10:harry potter and the deathly hallows: part 2 (2011)
11:what the #$*! do we know!? (a.k.a. what the bleep do we know!?) (2004)
12:harry potter and the deathly hallows: part 1 (2010)
13:harry potter and the half-blood prince (2009)
14:harry potter and the order of the phoenix (2007)
15:star wars: episode vi - return of the jedi (1983)
16:star wars: episode i - the phantom menace (1999)
17:dune (1984)
18:harry potter and the goblet of fire (2005)
19:black hole, the (1979)
20:greatest story ever told, the (1965)


In [65]:
predict([(245, 5), (693, 2)], 20, 0.2)

1:eyes wide shut (1999)
2:thin red line, the (1998)
3:a.i. artificial intelligence (2001)
4:saving private ryan (1998)
5:matrix reloaded, the (2003)
6:matrix revolutions, the (2003)
7:forrest gump (1994)
8:dances with wolves (1990)
9:braveheart (1995)
10:jfk (1991)
11:gladiator (2000)
12:matrix, the (1999)
13:cast away (2000)
14:bringing out the dead (1999)
15:vanilla sky (2001)
16:2001: a space odyssey (1968)
17:star wars: episode iii - revenge of the sith (2005)
18:nixon (1995)
19:godfather: part iii, the (1990)
20:castaway (1986)
